Let's start by importing BeautifulSoup to scrape the wikipedia page 

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
print('Libraries imported!')

Libraries imported!


Find the wikipedia page and open with BeautifulSoup

In [2]:
!wget -O filename.html en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
with open('filename.html') as html_file:
    soup= BeautifulSoup(html_file, 'lxml')

--2019-02-23 07:00:34--  http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:80... connected.
HTTP request sent, awaiting response... 301 TLS Redirect
Location: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M [following]
--2019-02-23 07:00:34--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80807 (79K) [text/html]
Saving to: ‘filename.html’

filename.html       100%[=====================>]  78.91K  --.-KB/s   in 0.04s  

2019-02-23 07:00:34 (1.73 MB/s) - ‘filename.html’ saved [80807/80807]



Parse wikipedia page for data

In [3]:
#Find the first table on page
table= soup.find('table', class_='wikitable sortable')

# Build parser
n_columns=0
n_rows=0
column_names= []

row_marker= 0 

# Find number of rows and columns
# also find columns header names 
for row in table.find_all('tr'):
    td_tags= row.find_all('td')
    if len(td_tags)>0:
        n_rows+=1
        if n_columns==0:
            n_columns== len(td_tags)
            
    #Handle column names
    th_tags= row.find_all('th')
    if len(th_tags)>0 and len(column_names)==0:
        for th in th_tags:
            column_names.append(th.get_text())

columns= column_names if len(column_names)>0 else range(0,n_columns)
df= pd.DataFrame(columns= columns, index= range(0, n_rows))
row_marker=0
for row in table.find_all('tr'):
    column_marker=0
    columns= row.find_all('td')
    for column in columns:
        df.iloc[row_marker, column_marker]= column.get_text()
        column_marker+=1
    if len(columns)>0:
        row_marker+=1

df.head()
        
        
            

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [4]:
# Table shape before cleaning operations
df.shape

(289, 3)

In [5]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is 'Not assigned'
df= df.drop(df[df.Borough == 'Not assigned'].index).reset_index(drop=True)
# Drop new line script(\n) from table 
df.replace('\n', '', regex=True, inplace=True)

df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [9]:
# Check type of Neighbourhood data
df.columns
df.rename(columns={'Neighbourhood\n': 'Neighbourhood'}, inplace=True)

In [10]:
# Merge rows with same borough
dfgrouped= df.groupby(['Postcode', 'Borough'], as_index=False, sort=False)['Neighbourhood'].apply(','.join).reset_index(name='Neighbourhood')
dfgrouped= pd.DataFrame(dfgrouped)
dfgrouped.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [11]:
# Where neighbourhood name is missing, apply borough name
dfgrouped['Neighbourhood'].loc[dfgrouped['Neighbourhood']=='Not assigned']= dfgrouped['Borough']
# Check if any neighbourhoods left with unassigned values
dfgrouped['Neighbourhood'].loc[dfgrouped['Neighbourhood']=='Not assigned'].count()

0

In [12]:
dfgrouped

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [13]:
dfgrouped.shape

(103, 3)

Let's add latitude and longitude coordinates for each neighbourhood

In [18]:
# open csv file 
!wget -O 'geospatial_data.csv' http://cocl.us/Geospatial_data
print('data downloaded!')

--2019-02-23 07:24:45--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-02-23 07:24:45--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-23 07:24:46--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-23 07:24:46--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjc

In [24]:
df_geo= pd.read_csv('geospatial_data.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [39]:
#check shape
df_geo.shape
df_geo.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

Great! the geospatial data from file matches the shape of our dataframe

Let's join the data from that file to our dataframe using Postal Codes as index

In [44]:
df_geo= df_geo.rename(columns={'Postal Code': 'Postcode'})
df_geo.columns

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')

In [47]:
df2= pd.merge(dfgrouped,
            df_geo[['Postcode','Latitude', 'Longitude']],
             on='Postcode')
df2

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
